In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.part_number'):
    shutil.rmtree('./file/1.part_number')

os.makedirs('./file/1.part_number')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import re

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    request_url = input_.loc[a, 'Url']
                    
                    resp = requests.get(request_url,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))
                        
                        type_ = html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0]
                        if type_.strip() == 'All' or re.sub(r' +', ' ', f'''{html.xpath('//select[@id="manufacturer_partsearch_007"]/option[@selected]/@value')[0]} {type_} Part Numbers''').strip() == re.sub(r' +', ' ', html.xpath('//div[@id="nav_o[1]"]/div[@class="inner"]/div/div[@style="display: inline-block; padding: 0em 5em 0em 5em; "]/text()')[0]).strip():
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if type_.strip() == 'All':
               raise

            # = = = = = = = = = = = = = = =

            group = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/@value')[0]

            # = = = = = = = = = = = = = = =

            list_dict = [json.loads(json_) for json_ in html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/input[contains(@id, "jsn[")]/@value')]

            # = = = = = = = = = = = = = = =

            list_url = [f'https://www.rockauto.com{href.strip()}' for href in html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/div[contains(@id, "nav_o[")]/div/table/tr/td[@class="nlabel nlbl-docolumns"]/a/@href')]

            # = = = = = = = = = = = = = = =

            if not list_url:
                raise

            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                                   'Type': type_,
                                   'Manufacturer': [dict_['catalogname'] for dict_ in list_dict],
                                   'Part Number': [dict_['partnumber'] for dict_ in list_dict],
                                   'Url': list_url,
                                   'Group': group,
                                   'Type Code': [dict_['parttype'] for dict_ in list_dict]})
            
            output.to_excel(f'''./file/1.part_number/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
            df_temp.loc[0, 'Request_Url'] = request_url
            
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.part_number/{file}',
              f'''./file/1.part_number/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[状态：ok，尝试次数：1] - 1.https://www.rockauto.com/en/parts/melling,timing+chain+&+component+kit,5756
[剩余数量：0] - [当前时间：19:02:36]

[状态：ok，尝试次数：1] - 2.https://www.rockauto.com/en/parts/motorcraft,fuel+tank+cap,5900
[剩余数量：0] - [当前时间：19:02:46]

输出ing...



Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3350.08it/s]

Done ~
